In [1]:
import pandas as pd
import numpy as np

In [8]:
judge_corpus = pd.read_csv('D:/期末專題/AI法官/rawData/forModel_mainContent.csv', encoding='utf-8')
judge_corpus.head()

,id,judge_court,judge_year,judge_month,judge_main_content,judge_main,judge_content,judge_multiCrimeCategory,judge_crimeDuration,judge_cooCrime,judge_recidivism,judge_available,judge_determine
0,4,臺灣彰化地方法院,111,1,主文:林燕輝犯汽車駕駛人無駕駛執照駕車過失傷害罪，處拘役參拾伍日，如易科罰金，以新臺幣壹仟元...,主文:林燕輝犯汽車駕駛人無駕駛執照駕車過失傷害罪，處拘役參拾伍日，如易科罰金，以新臺幣壹仟元...,犯罪事實及理由一、本案犯罪事實、證據，除犯罪事實欄一應補充「林燕輝所考領之自用小客車駕駛執照...,過失傷害,拘役參拾伍日,非共同犯罪,初犯,有效,受理
1,6,臺灣彰化地方法院,111,1,主文:黃鈵淞犯傷害人致重傷罪，處有期徒刑壹年捌月，緩刑參年，緩刑期間付保護管束，並應支付被害...,主文:黃鈵淞犯傷害人致重傷罪，處有期徒刑壹年捌月，緩刑參年，緩刑期間付保護管束，並應支付被害...,理由一、上揭犯罪事實，業據被告黃鈵淞於警詢、檢察官偵訊及本院審理中坦承不諱，核與證人即告訴人...,傷害,有期徒刑壹年捌月,非共同犯罪,初犯,有效,受理
2,8,臺灣彰化地方法院,111,1,主文:賴志忠共同傷害罪，處拘役伍拾日，如易科罰金，以新臺幣壹仟元折算壹日。犯罪事實及理由一、...,主文:賴志忠共同傷害罪，處拘役伍拾日，如易科罰金，以新臺幣壹仟元折算壹日。,犯罪事實及理由一、本案犯罪事實及證據，除下列事項外，餘均引用如附件檢察官聲請簡易判決處刑書之...,傷害,拘役伍拾日,共同犯罪,初犯,有效,受理
3,18,臺灣彰化地方法院,111,1,主文:王枷勛犯傷害罪，處有期徒刑陸月，如易科罰金，以新臺幣壹仟元折算壹日。理由一、以下本案所...,主文:王枷勛犯傷害罪，處有期徒刑陸月，如易科罰金，以新臺幣壹仟元折算壹日。,理由一、以下本案所引用被告以外之人於審判外之言詞或書面陳述，而不符合刑事訴訟法第159條之1...,傷害,有期徒刑陸月,非共同犯罪,初犯,有效,受理
4,19,臺灣彰化地方法院,111,1,主文:陳燕蓉犯傷害罪，處拘役伍拾日，如易科罰金，以新臺幣壹仟元折算壹日。洪玉芳無罪。理由一、...,主文:陳燕蓉犯傷害罪，處拘役伍拾日，如易科罰金，以新臺幣壹仟元折算壹日。洪玉芳無罪。,理由一、證據能力：㈠本院下述所引用被告以外之人於審判外之供述證據，檢察官及被告洪玉芳均同意作...,傷害,拘役伍拾日,非共同犯罪,初犯,有效,受理


`讀取 判決書文本`

In [9]:
content = judge_corpus.loc[:, 'judge_main_content']
with open('judge_corpus_ten.txt', 'w', encoding='utf-8') as file:
    for content in content:
        file.write(''.join(content) + '\n')

`結巴斷詞`

In [2]:
import jieba
import time
import re

# Tokenize
startTime = time.time()
with open('judge_corpus_ten_words.txt', 'w', encoding='utf-8') as new_f:
    with open('judge_corpus_ten.txt', 'r', encoding='utf-8') as f:
        
        # load 自訂義 dict
        dict = './dict_judge_update_new.txt'
        jieba.load_userdict(dict)
           
        # load 自訂義 stopword
        stopword = './stopword_judge_update.txt'
        stopwords = [line.strip() for line in open(stopword, 'r', encoding='utf-8').readlines()]
        
        for times, data in enumerate(f, 1):
            print('data num:', times)
            data = jieba.cut(data)
            data = [word for word in data if word != ' ']
            data = [word for word in data if word not in stopwords]  # 停用字
            data = ' '.join(data)  # 將 list 轉成 string

            new_f.write(data)
            
print(f"總花費時間 : {((time.time() - startTime)/60):.2f} 分鐘")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Jeh\AppData\Local\Temp\jieba.cache
Loading model cost 1.392 seconds.
Prefix dict has been built successfully.

KeyboardInterrupt



In [2]:
# 正則 stopword

import re
import time
import jieba

content = open('judge_corpus_ten.txt', 'r', encoding='utf-8')
content_ = list(content)
data_content = ''.join(content_)

# print(data_content)

r4 = "\【.*?】+|\《.*?》+|\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！\，。=？、：“”‘’￥……()《》【】]|[a-zA-Z]+|（|）|，|。|；|○|「|」"
data_content = re.sub(r4,'',data_content)
#print(type(data_content))

startTime = time.time()

with open('judge_corpus_ten_re.txt', 'w', encoding='utf-8') as new_f:
    tmp = []
    for a in data_content:
        tmp.append(a)
    new_content = ''.join(tmp)
    new_f.write(new_content)

    
print(f"總花費時間 : {((time.time() - startTime)/60):.2f} 分鐘")


總花費時間 : 0.29 分鐘


In [ ]:
# 正則後 結巴
import jieba
import time

startTime = time.time()
# with open('judge_corpus_ten_re_cut.txt', 'w', encoding='utf-8-sig') as new_f:
#      with open('judge_corpus_ten_re.txt', 'r', encoding='utf-8') as f:
with open('judge_corpus_ten_re_cut_2.txt', 'w', encoding='utf-8-sig') as new_f:
     with open('judge_corpus_ten_re.txt', 'r', encoding='utf-8') as f:
        #print(type(f))
        # load 自訂義 dict
        dict = './dict_judge_update_new.txt'
        jieba.load_userdict(dict)
           
        # load 自訂義 stopword
        stopword = './stopword_judge_update.txt'
        stopwords = [line.strip() for line in open(stopword, 'r', encoding='utf-8').readlines()]
        
        for times, data in enumerate(f, 1):
            print('data num:', times)
            data = jieba.cut(data)
            data = [word for word in data if word != ' ']
            data = [word for word in data if word not in stopwords]  # 停用字
            data = ' '.join(data)  # 將 list 轉成 string

            new_f.write(data)
            
print(f"總花費時間 : {((time.time() - startTime)/60):.2f} 分鐘") 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Jeh\AppData\Local\Temp\jieba.cache
Loading model cost 1.768 seconds.
Prefix dict has been built successfully.


data num: 1
data num: 2
data num: 3
data num: 4
data num: 5
data num: 6
data num: 7
data num: 8
data num: 9
data num: 10
data num: 11
data num: 12
data num: 13
data num: 14
data num: 15
data num: 16
data num: 17
data num: 18
data num: 19
data num: 20
data num: 21
data num: 22
data num: 23
data num: 24
data num: 25
data num: 26
data num: 27
data num: 28
data num: 29
data num: 30
data num: 31
data num: 32
data num: 33
data num: 34
data num: 35
data num: 36
data num: 37
data num: 38
data num: 39
data num: 40
data num: 41
data num: 42
data num: 43
data num: 44
data num: 45
data num: 46
data num: 47
data num: 48
data num: 49
data num: 50
data num: 51
data num: 52
data num: 53
data num: 54
data num: 55
data num: 56
data num: 57
data num: 58
data num: 59
data num: 60
data num: 61
data num: 62
data num: 63
data num: 64
data num: 65
data num: 66
data num: 67
data num: 68
data num: 69
data num: 70
data num: 71
data num: 72
data num: 73
data num: 74
data num: 75
data num: 76
data num: 77
data num

data num: 594
data num: 595
data num: 596
data num: 597
data num: 598
data num: 599
data num: 600
data num: 601
data num: 602
data num: 603
data num: 604
data num: 605
data num: 606
data num: 607
data num: 608
data num: 609
data num: 610
data num: 611
data num: 612
data num: 613
data num: 614
data num: 615
data num: 616
data num: 617
data num: 618
data num: 619
data num: 620
data num: 621
data num: 622
data num: 623
data num: 624
data num: 625
data num: 626
data num: 627
data num: 628
data num: 629
data num: 630
data num: 631
data num: 632
data num: 633
data num: 634
data num: 635
data num: 636
data num: 637
data num: 638
data num: 639
data num: 640
data num: 641
data num: 642
data num: 643
data num: 644
data num: 645
data num: 646
data num: 647
data num: 648
data num: 649
data num: 650
data num: 651
data num: 652
data num: 653
data num: 654
data num: 655
data num: 656
data num: 657
data num: 658
data num: 659
data num: 660
data num: 661
data num: 662
data num: 663
data num: 664
data n

data num: 1168
data num: 1169
data num: 1170
data num: 1171
data num: 1172
data num: 1173
data num: 1174
data num: 1175
data num: 1176
data num: 1177
data num: 1178
data num: 1179
data num: 1180
data num: 1181
data num: 1182
data num: 1183
data num: 1184
data num: 1185
data num: 1186
data num: 1187
data num: 1188
data num: 1189
data num: 1190
data num: 1191
data num: 1192
data num: 1193
data num: 1194
data num: 1195
data num: 1196
data num: 1197
data num: 1198
data num: 1199
data num: 1200
data num: 1201
data num: 1202
data num: 1203
data num: 1204
data num: 1205
data num: 1206
data num: 1207
data num: 1208
data num: 1209
data num: 1210
data num: 1211
data num: 1212
data num: 1213
data num: 1214
data num: 1215
data num: 1216
data num: 1217
data num: 1218
data num: 1219
data num: 1220
data num: 1221
data num: 1222
data num: 1223
data num: 1224
data num: 1225
data num: 1226
data num: 1227
data num: 1228
data num: 1229
data num: 1230
data num: 1231
data num: 1232
data num: 1233
data num: 

data num: 1715
data num: 1716
data num: 1717
data num: 1718
data num: 1719
data num: 1720
data num: 1721
data num: 1722
data num: 1723
data num: 1724
data num: 1725
data num: 1726
data num: 1727
data num: 1728
data num: 1729
data num: 1730
data num: 1731
data num: 1732
data num: 1733
data num: 1734
data num: 1735
data num: 1736
data num: 1737
data num: 1738
data num: 1739
data num: 1740
data num: 1741
data num: 1742
data num: 1743
data num: 1744
data num: 1745
data num: 1746
data num: 1747
data num: 1748
data num: 1749
data num: 1750
data num: 1751
data num: 1752
data num: 1753
data num: 1754
data num: 1755
data num: 1756
data num: 1757
data num: 1758
data num: 1759
data num: 1760
data num: 1761
data num: 1762
data num: 1763
data num: 1764
data num: 1765
data num: 1766
data num: 1767
data num: 1768
data num: 1769
data num: 1770
data num: 1771
data num: 1772
data num: 1773
data num: 1774
data num: 1775
data num: 1776
data num: 1777
data num: 1778
data num: 1779
data num: 1780
data num: 

...

data num: 25319
data num: 25320
data num: 25321
data num: 25322
data num: 25323
data num: 25324
data num: 25325
data num: 25326
data num: 25327
data num: 25328
data num: 25329
data num: 25330
data num: 25331
data num: 25332
data num: 25333
data num: 25334
data num: 25335
data num: 25336
data num: 25337
data num: 25338
data num: 25339
data num: 25340
data num: 25341
data num: 25342
data num: 25343
data num: 25344
data num: 25345
data num: 25346
data num: 25347
data num: 25348
data num: 25349
data num: 25350
data num: 25351
data num: 25352
data num: 25353
data num: 25354
data num: 25355
data num: 25356
data num: 25357
data num: 25358
data num: 25359
data num: 25360
data num: 25361
data num: 25362
data num: 25363
data num: 25364
data num: 25365
data num: 25366
data num: 25367
data num: 25368
data num: 25369
data num: 25370
data num: 25371
data num: 25372
data num: 25373
data num: 25374
data num: 25375
data num: 25376
data num: 25377
data num: 25378
data num: 25379
data num: 25380
data num

data num: 25832
data num: 25833
data num: 25834
data num: 25835
data num: 25836
data num: 25837
data num: 25838
data num: 25839
data num: 25840
data num: 25841
data num: 25842
data num: 25843
data num: 25844
data num: 25845
data num: 25846
data num: 25847
data num: 25848
data num: 25849
data num: 25850
data num: 25851
data num: 25852
data num: 25853
data num: 25854
data num: 25855
data num: 25856
data num: 25857
data num: 25858
data num: 25859
data num: 25860
data num: 25861
data num: 25862
data num: 25863
data num: 25864
data num: 25865
data num: 25866
data num: 25867
data num: 25868
data num: 25869
data num: 25870
data num: 25871
data num: 25872
data num: 25873
data num: 25874
data num: 25875
data num: 25876
data num: 25877
data num: 25878
data num: 25879
data num: 25880
data num: 25881
data num: 25882
data num: 25883
data num: 25884
data num: 25885
data num: 25886
data num: 25887
data num: 25888
data num: 25889
data num: 25890
data num: 25891
data num: 25892
data num: 25893
data num

data num: 26345
data num: 26346
data num: 26347
data num: 26348
data num: 26349
data num: 26350
data num: 26351
data num: 26352
data num: 26353
data num: 26354
data num: 26355
data num: 26356
data num: 26357
data num: 26358
data num: 26359
data num: 26360
data num: 26361
data num: 26362
data num: 26363
data num: 26364
data num: 26365
data num: 26366
data num: 26367
data num: 26368
data num: 26369
data num: 26370
data num: 26371
data num: 26372
data num: 26373
data num: 26374
data num: 26375
data num: 26376
data num: 26377
data num: 26378
data num: 26379
data num: 26380
data num: 26381
data num: 26382
data num: 26383
data num: 26384
data num: 26385
data num: 26386
data num: 26387
data num: 26388
data num: 26389
data num: 26390
data num: 26391
data num: 26392
data num: 26393
data num: 26394
data num: 26395
data num: 26396
data num: 26397
data num: 26398
data num: 26399
data num: 26400
data num: 26401
data num: 26402
data num: 26403
data num: 26404
data num: 26405
data num: 26406
data num

data num: 26858
data num: 26859
data num: 26860
data num: 26861
data num: 26862
data num: 26863
data num: 26864
data num: 26865
data num: 26866
data num: 26867
data num: 26868
data num: 26869
data num: 26870
data num: 26871
data num: 26872
data num: 26873
data num: 26874
data num: 26875
data num: 26876
data num: 26877
data num: 26878
data num: 26879
data num: 26880
data num: 26881
data num: 26882
data num: 26883
data num: 26884
data num: 26885
data num: 26886
data num: 26887
data num: 26888
data num: 26889
data num: 26890
data num: 26891
data num: 26892
data num: 26893
data num: 26894
data num: 26895
data num: 26896
data num: 26897
data num: 26898
data num: 26899
data num: 26900
data num: 26901
data num: 26902
data num: 26903
data num: 26904
data num: 26905
data num: 26906
data num: 26907
data num: 26908
data num: 26909
data num: 26910
data num: 26911
data num: 26912
data num: 26913
data num: 26914
data num: 26915
data num: 26916
data num: 26917
data num: 26918
data num: 26919
data num

data num: 27371
data num: 27372
data num: 27373
data num: 27374
data num: 27375
data num: 27376
data num: 27377
data num: 27378
data num: 27379
data num: 27380
data num: 27381
data num: 27382
data num: 27383
data num: 27384
data num: 27385
data num: 27386
data num: 27387
data num: 27388
data num: 27389
data num: 27390
data num: 27391
data num: 27392
data num: 27393
data num: 27394
data num: 27395
data num: 27396
data num: 27397
data num: 27398
data num: 27399
data num: 27400
data num: 27401
data num: 27402
data num: 27403
data num: 27404
data num: 27405
data num: 27406


`word2vec 模型訓練`

In [ ]:
seed          # 亂數種子
sg            # sg = 0 ( CBOW ) ; sg = 1 ( Skip-gram )
window_size   # 週圍的詞要看多少範圍
vector_size   # 轉成向量的維度
min_count     # 詞頻少於 min_count 的詞不會參與訓練
workers       # 訓練的並行數量
epochs        # 訓練的迭代次數 iter
batch_words   # 每次給予多少詞量訓練

In [78]:
from gensim.models import word2vec
import time
# train model

crime = ['crimeCategory_1_傷害直系血親尊親屬致死_jeba', # 0
         'crimeCategory_2_傷害直系血親尊親屬_jeba', 
         'crimeCategory_3_傷害致死_jeba', 
         'crimeCategory_4_過失傷害致重傷_jeba', 
         'crimeCategory_5_傷害致重傷_jeba', 
         'crimeCategory_6_重傷害未遂_jeba', 
         'crimeCategory_7_重傷致死_jeba', 
         'crimeCategory_8_重傷害_jeba',
         'crimeCategory_9_教唆傷害_jeba',
         'crimeCategory_10_肇事逃逸_jeba',  
         'crimeCategory_11_過失傷害_jeba',
         'crimeCategory_12_傷害_jeba',
         'crimeCategory_13_無罪_jeba'] # 12
crimeName = crime[0]

# Settings
seed = 666  
sg = 0  
window_size = 5  
vector_size = 150  
min_count = 1  
workers = 8  
epochs = 15  
batch_words = 1000  

print('開始 訓練 word2vec 模型')
startTime_train = time.time()
startTime_save = time.time()
startTime_total = time.time()

f'D:/期末專題/AI法官/rawData/crimeCategory/{crimeName}.txt'

train_data = word2vec.LineSentence('judge_corpus_ten_re_cut.txt')
model = word2vec.Word2Vec(
    train_data, 
    min_count=min_count,
    vector_size=vector_size,
    workers=workers,
    epochs=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)
print(f"train_model 花費時間 : {((time.time() - startTime_train)/60):.2f} 分鐘")

# word2vec_judge_sg_w5_v150_e15.model
# word2vec_judge_cbow_w5_v150_e15.model
# word2vec_judge_sg_w10_v300_e15.model

# word2vec_judge_cbow_w10_v300_e15.model

# word2vec_judge_cbow_w5_v150_e15_傷害直系血親尊親屬.model



# save model
print('儲存 word2vec 模型')
model.save('word2vec_judge_cbow_w5_v150_e15_傷害直系血親尊親屬.model')
print(f"save_model 花費時間 : {((time.time() - startTime_save)/60):.2f} 分鐘")

print(f"total 花費時間 : {((time.time() - startTime_total)/60):.2f} 分鐘")

開始 訓練 word2vec 模型
train_model 花費時間 : 8.98 分鐘
儲存 word2vec 模型
save_model 花費時間 : 9.00 分鐘
total 花費時間 : 9.00 分鐘


`讀取模型，找出相近字`

In [1]:
# load model
model = word2vec.Word2Vec.load('word2vec_judge_sg_w5_v150_e15.model')
model.wv.most_similar('傷害', topn=10)

[('重傷害', 0.6459618806838989),
 ('傷勢', 0.637721061706543),
 ('蕭等', 0.6189050674438477),
 ('之柳亦城', 0.6131632328033447),
 ('罪八', 0.6076111197471619),
 ('論科五核', 0.605745792388916),
 ('安東尼所', 0.6035436987876892),
 ('泰涉', 0.6010153889656067),
 ('吳鈞祥間', 0.5991635322570801),
 ('犯之性', 0.5987062454223633)]

`查看詞向量`

In [3]:
'''
拘役
有期徒刑
傷害直系血親尊親屬致死
傷害直系血親尊親屬
傷害致死
過失傷害致重傷
傷害致重傷
重傷害未遂
重傷致死
重傷害
教唆傷害
肇事逃逸
過失傷害
傷害
'''

# # 查看詞向量
model.wv['傷害']


# 查看最相近詞 positive / 最相近反義詞 negative  
model.wv.most_similar(positive=['傷害', '死亡'], negative=['被告'], topn=20)


# 計算相似程度
model.wv.similarity('傷害', '死亡')

array([-1.75784335e-01,  1.12871088e-01, -7.27776289e-02, -2.33424291e-01,
       -1.25480890e-01,  5.20821691e-01,  2.10063875e-01, -4.47833911e-02,
       -1.33698434e-01,  7.91313052e-01, -3.62404972e-01, -1.88534092e-02,
       -5.29249072e-01,  4.38722298e-02, -2.99450970e-04, -5.01029730e-01,
        2.00700104e-01,  1.83269858e-01,  1.82746917e-01,  9.29594487e-02,
        6.14310317e-02, -1.03493161e-01,  4.00503755e-01,  3.78722697e-01,
        4.42385316e-01,  4.23107177e-01, -1.11927077e-01, -6.53503716e-01,
        6.21326685e-01, -6.14006639e-01, -1.55657858e-01,  5.11004739e-02,
       -1.81991607e-01, -1.55860141e-01,  3.72491181e-01,  4.75262642e-01,
        2.48895749e-01,  3.55555922e-01,  1.71730250e-01,  4.54015955e-02,
       -3.38514030e-01,  2.63835676e-02, -5.24777889e-01, -3.05800378e-01,
        4.42887604e-01, -1.21810257e-01, -5.46004593e-01, -3.45320374e-01,
       -3.30256909e-01,  2.57032067e-01,  2.35473096e-01,  4.19446826e-01,
       -2.67656803e-01, -